In [ ]:
# In this notebook knowledge is built up step by step
# The code is commented to explain the steps

# Remaining tasks:
# error handling
# testing for speed

In [ ]:
# Simplest API test
import openai
from project_secrets import *   # API key stored in project_secrets.py

openai.api_key = OPENAI_API_KEY

response = openai.Completion.create(model="text-davinci-003", prompt="Say this is a test", temperature=0, max_tokens=7)
print(response.choices[0].text)

In [ ]:
# Define a function to get a response from the API
# Loop to get a range of responses via temperature

def get_response(prompt):
    for i in range(0, 11, 2):
        temperature = i / 10
        response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=temperature, max_tokens=7)
        print(f"Temperature: {temperature}.  Response: {response.choices[0].text}")

prompt="Say something like, 'this is only a test', but change the output"
get_response(prompt)

In [ ]:
# Parse & print JSON response to get temperature, text, logprobs, and usage data
import json

def get_response(prompt):
    for i in range(0, 11, 2):
        temperature = i / 10
        response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=temperature, max_tokens=7)
        data = response.to_dict()
        id = data['id']
        text = data['choices'][0]['text'].strip()
        logprobs = data['choices'][0]['logprobs']
        completion_tokens = data['usage']['completion_tokens']
        prompt_tokens = data['usage']['prompt_tokens']
        total_tokens = data['usage']['total_tokens']
        print(f"Response: {text}")
        print(f"\tTemperature:{temperature} ; Logprobs:{logprobs} ; Prompt tokens:{prompt_tokens} ; Total tokens:{total_tokens} ; Completion tokens:{completion_tokens}")

prompt="Say something like, 'this is only a test', but change the output"
get_response(prompt)

In [76]:
# Store the data in a dataframe
import pandas as pd

data = []

def get_response(prompt):
    for i in range(0, 11, 2):
       temperature = i / 10
       response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=temperature, max_tokens=7)
       new_temp = {"temp": temperature}
       response.update(new_temp)
       data.append(response)

prompt = "Express 'this is only a test' differently"
get_response(prompt)

normalized = pd.json_normalize(data)

df = pd.DataFrame()
df['id'] = normalized['id']
df['temperature'] = normalized['temp']
df['text'] = normalized['choices'][0][0]['text'].strip()
df['logprobs'] = normalized['choices'][0][0]['logprobs']
df['completion_tokens'] = normalized['usage.completion_tokens']
df['prompt_tokens'] = normalized['usage.prompt_tokens']
df['total_tokens'] = normalized['usage.total_tokens']

print(df)


                                   id  temperature                    text  \
0  cmpl-79FwdgPoaqnWAM0jNn3nSG8YizUqs          0.0  This is merely a trial   
1  cmpl-79Fwe4LALvg4OLPAyhyfBN96bGL1h          0.2  This is merely a trial   
2  cmpl-79FwfZmGvlDnduiG68QaO98BSM7zK          0.4  This is merely a trial   
3  cmpl-79Fwg8UPoQqE7lgiO4vmtMGbgOdKS          0.6  This is merely a trial   
4  cmpl-79Fwgajz5EjujArH9afKVTn49vEKS          0.8  This is merely a trial   
5  cmpl-79Fwh7fNRrejRy40JPbrcR9Rj4fAG          1.0  This is merely a trial   

  logprobs  completion_tokens  prompt_tokens  total_tokens  
0     None                  7              9            16  
1     None                  7              9            16  
2     None                  7              9            16  
3     None                  7              9            16  
4     None                  7              9            16  
5     None                  7              9            16  
